In [1]:
import nltk
import scipy
import pandas as pd
import numpy as np

import csv
from nltk.corpus import wordnet as wn
from libs import *

3.6635616461296463
0
[Synset('tiger.n.01')]
0
0
[Synset('tiger.n.01')]
11
11
[Synset('organism.n.01')]
11
11
[Synset('organism.n.01')]
0
0
[Synset('tiger.n.02')]


In [2]:
def main():
    word_sym=pd.read_csv('data\\WordSim353.csv')
    word_sym.insert(3,"wu_palmer",np.zeros(353))
    word_sym.insert(4,"shortest_path",np.zeros(353))
    word_sym.insert(5,"leak_chod",np.zeros(353))
    #print(word_sym)
    for i in range(0,353):
        v1 = word_sym.at[i,'Word 1']
        v2 = word_sym.at[i,'Word 2']
        max_wu=0
        max_sp=0
        max_lch=0
        for s1 in wn.synsets(v1):
            for s2 in wn.synsets(v2):
                actual_wu=wu_similarity(s1,s2)
                actual_sp=sp_similarity(s1,s2)
                actual_lch=lch_similarity(s1,s2)
                if actual_wu>max_wu:
                    max_wu=actual_wu
                if actual_sp>max_sp:
                    max_sp=actual_sp
                if actual_lch>max_lch:
                    max_lch=actual_lch
        word_sym.iat[i,3]= max_wu
        word_sym.iat[i,4]= max_sp
        word_sym.iat[i,5]= max_lch

    print(word_sym)


if __name__ == "__main__":
    main()



           Word 1    Word 2  Human (mean)  wu_palmer  shortest_path  leak_chod
0            love       sex          6.77      0.923           37.0   1.579784
1           tiger       cat          7.35      0.966           37.0   1.579784
2           tiger     tiger         10.00      1.000           38.0   1.591065
3            book     paper          7.46      0.875           36.0   1.278754
4        computer  keyboard          7.62      0.824           35.0   1.102662
..            ...       ...           ...        ...            ...        ...
348        shower     flood          6.03      0.636           34.0   0.977724
349       weather  forecast          8.34      0.133           25.0   0.465840
350      disaster      area          6.25      0.500           30.0   0.676694
351      governor    office          6.34      0.526           29.0   0.625541
352  architecture   century          3.78      0.308           29.0   0.625541

[353 rows x 6 columns]
